# DSL winter project 2023

### Bianco Matteo s300781, Grobbo Filippo s305723

## Import packages and modules

In [ ]:
import scipy.io.wavfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
from tqdm import tqdm
from scipy.signal import spectrogram
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from scipy.stats import skew, kurtosis
import noisereduce as nr
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler

## Import dataset

In [ ]:
df = pd.read_csv('dsl_data/development.csv')
df

In [ ]:
#Creation of new columns 'label' containing samples labels (concatenating columns 'action' and 'object')

df['label'] = df['action']+df['object']
df.drop(columns=['action','object'], inplace = True)

## Resampling

In [ ]:
#Counting cardinality of each class

np.unique(df['label'], return_counts=True)

In [ ]:
#Random oversampling

col_names = df.columns
y = df['label'].values
X = df.iloc[:,:-1].values
ros = RandomOverSampler(random_state=42, sampling_strategy='not majority') #resample all classes but the majority class
X_resampled, y_resampled = ros.fit_resample(X,y)
df_resampled = pd.DataFrame(np.hstack((X_resampled,y_resampled.reshape(-1,1))), columns=col_names)
df_resampled

In [ ]:
#Shuffle rows and reset indeces

df_resampled = df_resampled.sample(frac=1).reset_index(drop=True)

## Data preprocessing

In [ ]:
#Import audio wav files as numpy arrays

audio_list_res = []
sample_rates_res = []
for path in df_resampled['path']:
    file_read = scipy.io.wavfile.read(path)
    sample_rates_res.append(file_read[0])
    audio_list_res.append(file_read[1])

In [ ]:
#Check audio encoding

max_list_res=[]
for audio in audio_list_res:
    max_list_res.append(max(audio))
max(max_list_res)

In [ ]:
#Check sample rates different values

np.unique(sample_rates_res)

In [ ]:
#Compute mean absolute amplitude value of all audios (in a first moment we used this value as a trimming threshold)

mean = np.concatenate([np.abs(audio) for audio in audio_list_res]).mean()
mean  

In [ ]:
#Trimming audios to eliminate silence sections

threshold = 500  #we finally decided to use this threshold
#initialization of list and indeces
audio_trim=[]
drop_list=[]
i=0
for audio in tqdm(audio_list_res):  #scroll all audio in the list
    #initialization
    index_first=0
    index_last=-2
    #Find indeces of values above threshold
    indexes_above=np.where(np.abs(audio)>threshold)[0]
    #Check if there are values above the threshold
    if len(indexes_above)!=0:
        index_first=indexes_above[0]
        index_last=indexes_above[-1]
    else:
        drop_list.append(i)  #REM: audio made just of white noise ar not trimmed, we'll drop them later
    #i is the index of audios
    i=i+1
    #creation of trimmed audio
    audio_trim.append(audio[index_first:index_last+1])

In [ ]:
#Drop white noise audios

df_nonoise = df_resampled.drop(index=drop_list)

for index in sorted(drop_list, reverse=True):
    del audio_trim[index]
    del audio_list_res[index]
    del sample_rates_res[index]

In [ ]:
#Noise reduction of audios

audio_trim = [nr.reduce_noise(audio_trim[i], sample_rates_res[i], n_fft=1024, hop_length=256) for i in tqdm(range(len(audio_trim)))]

In [ ]:
#Compute Mel-spectrogram of each audio

audio_spectr = []
for audio, sr in tqdm(zip(audio_trim, sample_rates_res), total=len(audio_trim)):
    spectrogram = librosa.feature.melspectrogram(y=audio.astype(float), sr = sr, n_fft=1024, hop_length=256)
    #Converting power units in deciBel scale
    audio_spectr.append(librosa.power_to_db(spectrogram, ref=np.max))

In [ ]:
#Feature extraction from Mel-spectrograms

#number of blocks hyperparameters
num_col = 6  
num_row = 16
row_width = int(128/num_row)
num_chunks = num_col
#initialization of features matrix
df_arr = np.zeros((len(audio_spectr), 6*num_col*num_row))
#Loop on each Mel-spectrogram
for index, spectr in tqdm(enumerate(audio_spectr), total = len(audio_spectr)):
    #Initialization of matrices relative to each statistic to calculate for every spectrogram
    col_chunks = np.array_split(spectr, num_chunks, axis=1)
    features_m = np.zeros((num_row,num_col))
    features_s = np.zeros((num_row,num_col))
    features_max = np.zeros((num_row,num_col))
    features_min = np.zeros((num_row,num_col))
    features_1quart = np.zeros((num_row,num_col))
    features_3quart = np.zeros((num_row,num_col))
    #Loop on each block in which we divide the spectrogram
    for j, chunk in enumerate(col_chunks):
        for i in range(num_row):
            sub_mat = chunk[row_width*i:row_width*(i+1),:]
            #Computation of statistics
            features_m[i,j] = sub_mat.mean()
            features_s[i,j] = sub_mat.std()
            features_max[i,j] = sub_mat.max()
            features_min[i,j] = sub_mat.min()
            features_1quart[i,j] = np.quantile(sub_mat.reshape(-1), q=0.25)
            features_3quart[i,j] = np.quantile(sub_mat.reshape(-1), q=0.75)
    #Grouping features together
    features = np.concatenate((features_m.reshape(-1), features_s.reshape(-1), features_max.reshape(-1), features_min.reshape(-1), features_1quart.reshape(-1), features_3quart.reshape(-1)))
    #Assigning features relative to each audio to the feature matrix
    df_arr[index,:] = features.reshape(1,-1)

In [ ]:
#Creation of DataFrame containing features extracted from Mel-spectrograms

#Define columns names
df_col=list(df_resampled.columns)
for j in range(num_col):
    for i in range(num_row):
        df_col=df_col+[f'mean_{i,j}']+[f'std_{i,j}']+[f'max_{i,j}']+[f'min_{i,j}']+[f'1quart_{i,j}']+[f'3quart_{i,j}']

df_spectr = pd.DataFrame(index = range(len(df_nonoise.index)), columns=df_col)
#We assign to each sample also the fetures contained in the original dataframe
df_spectr.iloc[:,:9] = df_nonoise.values
#We assign to each sample the new features computed on Mel-spectrograms
df_spectr.iloc[:,9:] = df_arr
df_spectr.columns = df_col

df_spectr

In [ ]:
#Add feature with time duration for each audio

df_spectr['duration'] = [len(audio)/rate for audio, rate in zip(audio_trim, sample_rates_res)]

In [ ]:
#One-hot Encoding of feature "gender"

df_encoded = pd.get_dummies(df_spectr, columns=list(df_spectr.columns[[6]]))

In [ ]:
#Modify col name of feature 'Self-reported fluency level' beacuse it gives issues in accessing it

col_names = list(df_encoded.columns)
col_names[3] = 'Self reported fluency level'
df_encoded.columns = col_names

In [ ]:
#Check unique values of categorical features 'Self reported fluency level' and 'ageRange'

df_encoded['Self reported fluency level'].unique(), df_encoded['ageRange'].unique()

In [ ]:
#Label encoding of features 'Self reported fluency level' and 'ageRange'

df_encoded.replace({'Self reported fluency level': {'basic':1, 'intermediate':2, 'advanced':3, 'native':4},\
                    'ageRange':{'22-40':1, '41-65':2, '65+':3}},\
                   inplace=True)

In [ ]:
#Drop and save in another variable the column containing audio labels

label = df_encoded['label']

#Drop features which resulted with low feature importance (with Random Forest) in preliminary analysis

df_encoded.drop(columns=['Id', 'path', 'speakerId', 'First Language spoken', 'Current language used for work/school','label'], inplace=True)
df_encoded

## Grid seach for model selection and hyperparameters tuning

In [ ]:
#Feature normalization for SVM (run only for SVM)

scaler_SVM = StandardScaler()
X_scaled = scaler_SVM.fit_transform(df_encoded.values)

In [ ]:
#Gridsearch with SVM

clf = SVC(random_state=42)
params = {'C':(1,5,10,50), 'degree':(3,5,7), 'kernel':('poly', 'rbf')}
gs_over = GridSearchCV(estimator = clf, param_grid = params, scoring = 'accuracy', verbose=10)
gs_over.fit(X_scaled, label)
pd.DataFrame(gs_over.cv_results_).sort_values(by=['rank_test_score'])

In [ ]:
#Save in csv file results of SVM grid search

gs_over_df = pd.DataFrame(gs_over.cv_results_).sort_values(by=['rank_test_score'])
gs_over_df.to_csv(path_or_buf='grid_search_over_16rows_minmax.csv')

In [ ]:
#Gridsearch with RandomForest

clf = RandomForestClassifier(random_state=42)
params = {'n_estimators':(160,170,180), 'max_samples':(0.7,0.8), 'min_samples_split':(2,4)}
gs_rf = GridSearchCV(estimator = clf, param_grid = params, scoring = 'accuracy', verbose=10)
gs_rf.fit(df_encoded.values, label)
pd.DataFrame(gs_rf.cv_results_).sort_values(by=['rank_test_score'])

In [ ]:
#Save in csv file results of Random Forest grid search

gs_over_df = pd.DataFrame(gs_rf.cv_results_).sort_values(by=['rank_test_score'])
gs_over_df.to_csv(path_or_buf='grid_search_over_RF_32rows.csv')

## Test best SVM using larger values for hyperparameter C

In [ ]:
#Gridsearch with best SVM to test if bigger values of C may improve performances

clf = SVC(kernel = 'rbf', random_state=42)
params = {'C':(100, 200, 500)}
gs_over = GridSearchCV(estimator = clf, param_grid = params, scoring = 'accuracy', verbose=10)
gs_over.fit(X_scaled, label)
pd.DataFrame(gs_over.cv_results_).sort_values(by=['rank_test_score'])

## Plots for the report

In [ ]:
#Import audio wav files from the original dataset

audio_list = []
sample_rates = []
for path in df['path']:
    file_read = scipy.io.wavfile.read(path)
    sample_rates.append(file_read[0])
    audio_list.append(file_read[1])

In [ ]:
#Histogram of time durations of audios

duration = []
for audio, rate in zip(audio_list, sample_rates):
    duration.append(len(audio)/rate)
fig, ax = plt.subplots(figsize=(7,7))
ax.hist(duration, bins=50)
ax.set_xlabel('Duration (s)')
ax.set_ylabel('Number of audios')
plt.grid()
plt.show()
#plt.savefig('duration_plot.png')

In [ ]:
#Look for a proper audio file to use for the next plot 

index = duration.index(20, 616)
index

In [ ]:
#Plot the chosen audio in time domain

fig, ax = plt.subplots(figsize=(7,7))
ax.plot(audio_list[index])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude (16-bit encoding)')
ax.set_xticks([i*sample_rates[index] for i in range(0,25,5)])
ax.set_xticklabels([0,5,10,15,20])
plt.grid()
plt.show()
#plt.savefig('audio_20s.png')

In [ ]:
#Find where the chosen audio is trimmed, for the next plot

threshold = 500
audio = audio_list[index]
index_first=0
index_last=-2
#first index
indexes_above=np.where(np.abs(audio)>threshold)[0]
if len(indexes_above)!=0:
    index_first=indexes_above[0]
    index_last=indexes_above[-1]
else:
    drop_list.append(i) 
    #creation of trimmed audio
audio_trimmed = audio[index_first:index_last+1] #We also trim the audio for spectrogram plot

In [ ]:
#Plot chosen audio in time domain, adding lines showing where it is trimmed

fig, ax = plt.subplots(figsize=(7,7))
ax.plot(audio_list[index])
ax.vlines([index_first, index_last], ymin=min(audio_list[index]), ymax=max(audio_list[index]), colors='red', linestyles='dashed', label='Where audio is cut')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude (16-bit encoding)')
ax.set_xticks([i*sample_rates[index] for i in range(0,25,5)])
ax.set_xticklabels([0,5,10,15,20])
ax.legend()
plt.grid()
plt.show()
#plt.savefig('audio_trim.png')

In [ ]:
#Apply noise reduction and compute Mel-spectrogram of the chosen audio

audio_nr = nr.reduce_noise(audio_trimmed, sample_rates[index], n_fft=1024, hop_length=256)
spectrogram = librosa.feature.melspectrogram(y=audio_nr.astype(float), sr = sample_rates[index], n_fft=1024, hop_length=256)
audio_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)

In [ ]:
#Plot Mel-spectrogram of the chosen audio

fig, ax = plt.subplots()
img = librosa.display.specshow(audio_spectrogram, x_axis='time',
                               y_axis='mel', sr=sample_rates[index], ax=ax)
fig.colorbar(img, ax=ax, format='%+2.0f dB')
plt.show()
#plt.savefig('Mel_spectrogram.png')

## Applying all preprocessing steps to evaluation set for submission

In [ ]:
#Import evaluation set

df_eval = pd.read_csv('dsl_data/evaluation.csv')
df_eval

In [ ]:
#Import audio files of evaluation set

audio_list_eval = []
sample_rates_eval = []
for path in df_eval['path']:
    file_read = scipy.io.wavfile.read(path)
    sample_rates_eval.append(file_read[0])
    audio_list_eval.append(file_read[1])

In [ ]:
#Applying trimming procedure

threshold=500
audio_trim_eval=[]
drop_list_eval=[]
i=0
for audio in tqdm(audio_list_eval):
    index_first=0
    index_last=-2
    #first index
    indexes_above=np.where(np.abs(audio)>threshold)[0]
    if len(indexes_above)!=0:
        index_first=indexes_above[0]
        index_last=indexes_above[-1]
    else:
        drop_list_eval.append(i)
    #i is the index of audios
    i=i+1
    #creation of trimmed audio
    audio_trim_eval.append(audio[index_first:index_last+1])

In [ ]:
#Here we see that on the evaluation set no audio is dropped (what happens to audio which are made of only noise)

drop_list_eval

In [ ]:
#Noise reduction on evaluation set

audio_trim_eval = [nr.reduce_noise(audio_trim_eval[i], sample_rates_eval[i], n_fft=1024, hop_length=256) for i in tqdm(range(len(audio_trim_eval)))]

In [ ]:
#Spectrograms calculation of evaluation set

audio_spectr_eval = []
for audio, sr in tqdm(zip(audio_trim_eval, sample_rates_eval), total=len(audio_trim_eval)):
    spectrogram = librosa.feature.melspectrogram(y=audio.astype(float), sr = sr, n_fft=512, hop_length=256)
    audio_spectr_eval.append(librosa.power_to_db(spectrogram, ref=np.max))

In [ ]:
#Features extraction from evaluation set spectrograms

num_col = 6
num_row = 16
row_width = int(128/num_row)
num_chunks = num_col
df_arr_eval = np.zeros((len(audio_spectr_eval), 6*num_col*num_row))
for index, spectr in tqdm(enumerate(audio_spectr_eval), total = len(audio_spectr_eval)):
    col_chunks = np.array_split(spectr, num_chunks, axis=1)
    features_m = np.zeros((num_row,num_col))
    features_s = np.zeros((num_row,num_col))
    features_max = np.zeros((num_row,num_col))
    features_min = np.zeros((num_row,num_col))
    features_1quart = np.zeros((num_row,num_col))
    features_3quart = np.zeros((num_row,num_col))
    for j, chunk in enumerate(col_chunks):
        for i in range(num_row):
            sub_mat = chunk[row_width*i:row_width*(i+1),:]
            features_m[i,j] = sub_mat.mean()
            features_s[i,j] = sub_mat.std()
            features_max[i,j] = max(sub_mat.reshape(-1))
            features_min[i,j] = min(sub_mat.reshape(-1))
            features_1quart[i,j] = np.quantile(sub_mat.reshape(-1), q=0.25)
            features_3quart[i,j] = np.quantile(sub_mat.reshape(-1), q=0.75)
    features = np.concatenate((features_m.reshape(-1), features_s.reshape(-1), features_max.reshape(-1), features_min.reshape(-1), features_1quart.reshape(-1), features_3quart.reshape(-1)))
    df_arr_eval[index,:] = features.reshape(1,-1)

In [ ]:
#Definition of a dataset containing evaluation set features

df_col_eval=list(df_eval.columns)
for j in range(num_col):
    for i in range(num_row):
        df_col_eval=df_col_eval+[f'mean_{i,j}']+[f'std_{i,j}']+[f'max_{i,j}']+[f'min_{i,j}']+[f'1quart_{i,j}']+[f'3quart_{i,j}']       
            
df_spectr_eval = pd.DataFrame(index = range(len(df_eval.index)), columns=df_col_eval)
df_spectr_eval.iloc[:,:8] = df_eval.values
df_spectr_eval.iloc[:,8:] = df_arr_eval

df_spectr_eval

In [ ]:
#Add feature with time duration

df_spectr_eval['duration'] = [len(audio)/rate for audio, rate in zip(audio_trim_eval, sample_rates_eval)]

In [ ]:
#One-hot encoding of feature 'gender'

df_enceval = pd.get_dummies(df_spectr_eval, columns=list(df_spectr_eval.columns[[6]]))

In [ ]:
#Modify name of column 'Self-reported fluency level'

col_names = list(df_enceval.columns)
col_names[3] = 'Self reported fluency level'
df_enceval.columns = col_names

In [ ]:
#Label encoding of features 'Self reported fluency level' and 'ageRange'
 
df_enceval.replace({'Self reported fluency level': {'basic':1, 'intermediate':2, 'advanced':3, 'native':4},\
                    'ageRange':{'22-40':1, '41-65':2, '65+':3}},\
                   inplace=True)

In [ ]:
#Drop features which resulted with low feature importance (with Random Forest) in preliminary analysis

df_enceval.drop(columns=['Id', 'path', 'speakerId', 'First Language spoken', 'Current language used for work/school'], inplace=True)

In [ ]:
#Normalize data according to scale of training set (only for SVM)

X_eval_scaled = scaler_SVM.transform(df_enceval.values)

In [ ]:
#Submission with Random Forest grid search best result (obtained on training set)

y_pred = gs_rf.best_estimator_.predict(df_enceval.values)

In [ ]:
#Submission with SVM grid search best result (obtained on training set)

y_pred = gs_over.best_estimator_.predict(X_eval_scaled)

In [ ]:
#Creation of dataframe with predicted labels

ris = pd.DataFrame({'Id':df_enceval.index, 'Predicted': y_pred})

#Save dataframe to csv file (in order to submit to leaderboard)

ris.to_csv(path_or_buf='result_exam23.csv', index=False)